<a href="https://colab.research.google.com/github/n-carcione/16664_perception_proj/blob/main/proj_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title mount your Google Drive
#@markdown Your work will be stored in a folder called `hw_16831` by default to prevent Colab instance timeouts from deleting your edits.

import os
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
#@title set up mount symlink

DRIVE_PATH = '/content/gdrive/My\ Drive/16664_percep_proj'
DRIVE_PYTHON_PATH = DRIVE_PATH.replace('\\', '')
if not os.path.exists(DRIVE_PYTHON_PATH):
  %mkdir $DRIVE_PATH

## the space in `My Drive` causes some issues,
## make a symlink to avoid this
SYM_PATH = '/content/16664_percep_proj'
if not os.path.exists(SYM_PATH):
  !ln -s $DRIVE_PATH $SYM_PATH

In [3]:
%cd /content/gdrive/My Drive/16664_percep_proj
!apt update 
!apt install -y --no-install-recommends \
        build-essential \
        curl \
        git \
        gnupg2 \
        make \
        cmake \
        ffmpeg \
        swig \
        libz-dev \
        unzip \
        zlib1g-dev \
        libglfw3 \
        libglfw3-dev \
        libxrandr2 \
        libxinerama-dev \
        libxi6 \
        libxcursor-dev \
        libgl1-mesa-dev \
        libgl1-mesa-glx \
        libglew-dev \
        libosmesa6-dev \
        lsb-release \
        ack-grep \
        patchelf \
        wget \
        xpra \
        xserver-xorg-dev \
        xvfb \
        python-opengl \
        ffmpeg > /dev/null 2>&1

/content/gdrive/My Drive/16664_percep_proj
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
25 packages can be upgraded. Run 'apt list --upgradable' to see them.


In [4]:
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install csv
%pip install torch
%pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
%cd /content/gdrive/My Drive/16664_percep_proj
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torchvision.transforms import transforms

class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index,0]+"_image.jpg")
        image = plt.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index,1]))

        if self.transform:
            image = self.transform(image)

        return(image, y_label)

/content/gdrive/My Drive/16664_percep_proj


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.autograd import Variable

# Example architecture
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        # Define the network layer structure here
        # Example from the website shown here
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(12)
        self.pool = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(24)
        self.conv4 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(24)
        self.fc1 = nn.Linear(24*520*951, 3)

    def forward(self, input):
        # Define how a forward pass through the network would look here
        # This function contains all the nonlinear functions between layers
        # The example from the website is continued here
        output = F.relu(self.bn1(self.conv1(input)))
        output = F.relu(self.bn2(self.conv2(output)))
        output = self.pool(output)
        output = F.relu(self.bn3(self.conv3(output)))
        output = F.relu(self.bn4(self.conv4(output)))
        output = output.view(-1, 24*520*951)
        output = self.fc1(output)

        return output

# # Architecture from the slides
# class Network(nn.Module):
#     def __init__(self):
#         super(Network, self).__init__()
#         # Define the network layer structure here
#         # Example from the slides shown here
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=1)
#         self.bn1 = nn.BatchNorm2d(12)
#         self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
#         self.bn2 = nn.BatchNorm2d(12)
#         self.pool1 = nn.MaxPool2d(2,2)
#         self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
#         self.bn3 = nn.BatchNorm2d(24)
#         self.conv4 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
#         self.bn4 = nn.BatchNorm2d(24)
#         self.pool2 = nn.MaxPool2d(2,2)
#         self.conv5 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
#         self.bn5 = nn.BatchNorm2d(24)
#         self.conv6 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
#         self.bn6 = nn.BatchNorm2d(24)
#         self.pool3 = nn.MaxPool2d(2,2)
#         self.fc1 = nn.Linear(24*128*235, 3)

#     def forward(self, input):
#         # Define how a forward pass through the network would look here
#         # This function contains all the nonlinear functions between layers
#         output = F.relu(self.bn1(self.conv1(input)))
#         output = F.relu(self.bn2(self.conv2(output)))
#         output = self.pool1(output)
#         output = F.relu(self.bn3(self.conv3(output)))
#         output = F.relu(self.bn4(self.conv4(output)))
#         output = self.pool2(output)
#         output = F.relu(self.bn5(self.conv5(output)))
#         output = F.relu(self.bn6(self.conv6(output)))
#         output = self.pool3(output)
#         output = output.view(-1, 24*128*235)
#         output = self.fc1(output)

#         return output

In [7]:
import os
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.autograd import Variable

from torchvision.transforms import transforms
from torch.utils.data import DataLoader

batch_size = 10
learning_rate = 0.001
weight_decay = 0.0001

%cd /content/gdrive/My Drive/16664_percep_proj
csv_file = os.path.join(os.getcwd(), "trainval/labels.csv")
root_dir = os.path.join(os.getcwd(), "trainval")
transformations = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
training_data = CustomDataset(csv_file=csv_file, root_dir=root_dir, transform=transformations)
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)

model = Network()
# path = os.path.join(os.getcwd(), "trained_model.pth")
# model.load_state_dict(torch.load(path))
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

def saveModel():
    print("Saving model!!")
    path = os.path.join(os.getcwd(), "./trained_model.pth")
    torch.save(model.state_dict(), path)

def train(num_epochs):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("The model willbe running on ", device, " device")
    model.to(device)

    for epoch in range(num_epochs):
        running_loss = 0.0

        for i, (images, labels) in enumerate(train_dataloader, 0):
            # get the inputs
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))

            # zero the parameter gradients
            optimizer.zero_grad()
            # predict the classes using images from the training set
            outputs = model(images)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, labels)
            # backprop the loss
            loss.backward()
            # adjust the parameters based on the calculated gradients
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/100))
                running_loss = 0.0
                saveModel()

train(5)
print("Finished training")
saveModel()

/content/gdrive/My Drive/16664_percep_proj
The model willbe running on  cuda:0  device
[1,   100] loss: 63.445
Saving model!!
[1,   200] loss: 23.141
Saving model!!
[1,   300] loss: 16.958
Saving model!!
[1,   400] loss: 11.735
Saving model!!
[1,   500] loss: 6.509
Saving model!!
[1,   600] loss: 4.898
Saving model!!
[1,   700] loss: 2.793
Saving model!!
[2,   100] loss: 1.960
Saving model!!
[2,   200] loss: 1.623
Saving model!!
[2,   300] loss: 2.040
Saving model!!
[2,   400] loss: 1.432
Saving model!!
[2,   500] loss: 1.523
Saving model!!
[2,   600] loss: 1.159
Saving model!!
[2,   700] loss: 1.018
Saving model!!
[3,   100] loss: 0.719
Saving model!!
[3,   200] loss: 0.651
Saving model!!
[3,   300] loss: 0.708
Saving model!!
[3,   400] loss: 0.828
Saving model!!
[3,   500] loss: 0.701
Saving model!!
[3,   600] loss: 0.605
Saving model!!
[3,   700] loss: 0.776
Saving model!!
[4,   100] loss: 0.635
Saving model!!
[4,   200] loss: 0.536
Saving model!!
[4,   300] loss: 0.491
Saving model

In [8]:
from torchvision.transforms import transforms
from glob import glob
import csv
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch

# Loading the model trained above
model = Network()
path = os.path.join(os.getcwd(), "trained_model.pth")
model.load_state_dict(torch.load(path))
transformations = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Get all the test image file locations
%cd /content/gdrive/My Drive/16664_percep_proj
sub_csv_file = os.path.join(os.getcwd(), "submission.csv")
root_dir = os.path.join(os.getcwd(), "test")
test_files = glob(os.path.join(root_dir, "*/*_image.jpg"))

# Trim the file names to just the folder and image number to match expected csv format
trimmed_files = [s.replace(os.getcwd(),'') for s in test_files]
trimmed_files = [s.replace('/test/','') for s in trimmed_files]
trimmed_files = [s.replace('_image.jpg','') for s in trimmed_files]

# Pass each test file through the trained network and record the predicted label
predictions = []
i = 0
for test_file in test_files:
  img = plt.imread(test_file)
  input = transformations(img)
  input = input[None, :]
  output = model(input)
  _, predicted = torch.max(output, 1)
  predictions.append(predicted.item())
  if i % 50 == 0:
    print(i)
  i += 1

# Create the csv file, including the expected header
header = ['guid/image', 'label']
data = [[file, pred] for (file, pred) in zip(trimmed_files, predictions)]
with open(sub_csv_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)

/content/gdrive/My Drive/16664_percep_proj
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
